## 🧾 Bank Record Categorization System
### Bank Records Analyst (bank_record_analyst.ipynb) 
###### Note - it superceded categorize_monthly_expenses.ipynb

An application to categorize, summarize, and analyze checking and credit card bank records. It provides a modular framework for classifying bank transactions into semantic categories and subcategories using pattern-based matching. It supports both human-readable summaries and programmatic filtering for financial analysis.

---
### Reads input files from folder: ./data 
#### DATA NAMING CONVENTION:  2025-08-A&T-CheckingAcct.xlsx / 2025-08-A&T-CCard.xlsx

---

### 📂 Data Structures

#### `categories_to_subcategories_tree: dict[str, dict[str, dict]]`
Defines the hierarchical taxonomy of financial categories and their subcategories.

- **Top-level keys** represent broad financial domains (e.g. `'INCOME'`, `'FOOD'`, `'HOUSING'`).
- **Nested keys** represent specific subcategories (e.g. `'Anita Income'`, `'Dining Out'`).
- Leaf nodes are empty dicts, to keep the datastructure uncluttered and human readible
- Leaf nodes are programatically populated with patterns from the second datastructure, allowing flexibility and extensibility.

**Example:**
```python
'FOOD': {
    'Groceries': {},
    'Dining Out': {},
    'Fast Food': {}
}
```

#### `subcategories_to_patterns: dict[str, list[str | re.Pattern]]`
Maps subcategories to lists of string or regex patterns used to identify matching bank record descriptions.
- **Patterns** may be **simple substrings** or **compiled regular expressions**.
- Enables flexible matching across diverse transaction formats.

**Example:**
```python
'Bills n Utilities': [
    'VERIZON',
    'DOMINION',
    re.compile(r'ATT\*BILL')
]
```



In [ ]:
categories_to_subcategories_tree = {
    'INCOME': {
        'Anita Income': {},
        'Fidelity Transfer': {},
        'KeyBank Cash-Back': {}
    },
    'TAXES': {
        'Taxes': {}
    },
    'FEES': {
        'Transaction Fees': {}
    },
    'EXCLUDE': {
        'Visa Payment': {},
        'Visa Payment Received': {}
    },
    'HOUSING': {
        'Mortgage': {},
        'Bills n Utilities': {}
    },
    'INSURANCE': {
        'Medical Insurance': {},
        'Car Insurance': {}
    },
    'HEALTHCARE': {
        'Medical and Dental': {},
        'Pharmacy': {}
    },
    'EDUCATION': {
        'College Tuition': {},
        'Art Supplies': {}
    },
    'PROFESSIONAL': {
        'Professional Fees': {},
        'AI API charges': {},
        'Professional Services': {},
        'Liability Insurance': {}
    },
    'TRANSPORTATION': {
        'Car Registration': {},
        'Gas': {},
        'Parking and Tolls': {},
        'Car Maintenance': {}
    },
    'FOOD': {
        'Groceries': {},
        'Dining Out': {},
        'Fast Food': {},
        'World Food': {},
        'Why DOORDASH?': {}
    },
    'SELFCARE & WELLBEING': {
        'Aikido n Yoga': {},
        'Beauty n Supplies': {},
        'Sound Bath': {}
    },
    'HOME & GARDEN': {
        'House Maintenance': {},
        'Furnishing': {},
        'Garden': {}
    },
    'SUBSCRIPTIONS': {
        'Subscription': {}
    },
    'SHOPPING': {
        'Amazon': {},
        'Department Store': {},
        'Clothes': {},
        'Kindle n Books': {},
        'Software n Accessories': {},
        'Gifts': {},
        'Kids Toys': {},
        'Cycling n Paddling': {}
    },
    'PETS': {
        'Cat Food n Supplies': {},
        'Cat Health': {}
    },
    'ENTERTAINMENT': {
        'Fun Out': {},
        'Movies n Theater': {},
        'Music n Games': {},
        'Memberships': {}
    },
    'MISCELLANEOUS': {
        'Political Donations': {},
        'ATM Wthdrw n Dpsit': {},
        'Vending Machines': {},
        'Parcels': {}
    },
    'VACATIONS & TRAVEL': {
        'Air Travel': {},
        'Visiting Grandma Ela': {},
        'Vacation SC': {},
        'Visiting Wanda': {},
        'Visiting Eva': {}
    }
}

In [ ]:
import re   # regular expressions for complex patterns

subcategories_to_patterns = {
    'Anita Income': [
        'ZELLE DEP ANITA'
    ],
    'Fidelity Transfer': [
        'FID '
    ],
    'KeyBank Cash-Back': [
        'KEY REWARDS',
        'GIFT FROM KEY BANK'
    ],
    'Taxes': [
        'TAXREFUND',
        ' IRS ',
        'TAX REF',
        'RITA',
        'CHECK # 746',
        'CHECK # 747'
    ],
    'Transaction Fees': [
        'TRANSACTION FEE'
    ],
    'Visa Payment': [
        'INTERNET TRF TO CCA'
    ],
    'Visa Payment Received': [
        'PAYMENT RECEIVED'
    ],
    'Mortgage': [
        'WFHM'
    ],
    'Bills n Utilities': [
        'VERIZON',
        'VZWRLSS',
        'DOMINION',
        'FIRST ENERGY',
        'NORTHEAST OHIO',
        'CLEVELAND HEIGHTS',
        'ENBRIDGE GAS',
        'ATT ',
        'ATT*BILL',
        'NEORSD',
        'CWD'
    ],
    'Medical Insurance': [
        'MEDICARE',
        'VSP',
        'UNITEDHEALTHCARE',
        'ROCKWELL',
        'AARP HEALTH',
        'DELTA DENTAL'
    ],
    'Car Insurance': [
        'LIBERTY MUTUAL'
    ],
    'Medical and Dental': [
        'PEDIATRICS',
        'CLEVELAND CLINIC',
        'METROHEALTH',
        'WESTERN RESERVE PERIO',
        'HILLCREST ',
        'CLEVELAND KIDNEY ',
        'SPRY SENIOR',
        'ETNA CLEVELAND',
        'HEIGHTS DENTAL '
    ],
    'Pharmacy': [
        'CVS',
        'WALGREENS'
    ],
    'College Tuition': [
        'SMARTPAYCIA',
        'CASHNET',
        'CAMPUS CIA',
        'BURREN',
        'COLLEGE'
    ],
    'Art Supplies': [
        'UTRECHT',
        ' ART '
    ],
    'Professional Fees': [
        'LICENSURE',
        'LICENSE',
        'CENTER FOR INTUITIVE'
    ],
    'AI API charges': [
        'OPENAI '
    ],
    'Professional Services': [
        'PAUKENLEGAL'
    ],
    'Liability Insurance': [
        'CPH LIABILITY'
    ],
    'Car Registration': [
        'BUREAU MOTOR VE'
    ],
    'Gas': [
        'SUNOCO',
        'BP',
        'SHELL',
        'MARATHON',
        'CIRCLE K',
        'SHEETZ',
        'GAS',
        'SPEEDWAY'
    ],
    'Parking and Tolls': [
        'EZ PASS REAL TIME',
        'GARAG ',
        'PARKING CLEVELAND'
    ],
    'Car Maintenance': [
        'REPAIR',
        'AUTO',
        'AUTO BODY',
        'QUALITY AUTO'
    ],
    'Groceries': [
        'GROCERY',
        'HEINEN',
        'DAVE',
        'WHOLE',
        'SODA',
        'TRADE'
    ],
    'Dining Out': [
        'TAVERN',
        'TOMMYS',
        'CAFE',
        'WASABI',
        'PACIFIC',
        'ANATOLIA',
        'BATUQUI',
        'PHO',
        'LAKE HOUSE',
        'DEWEY',
        'MAROTTA ',
        'BANANA',
        'BANGKOK',
        'HIBACHI',
        'BRASSICA',
        'RESTAUR',
        'BUFFALO',
        'COZUMEL',
        'FIRST WATCH',
        'PARADISE BIRYANI',
        'CARIBOU COFFEE',
        'STONE OVEN',
        'SEOUL GARDEN ',
        'YOURS TRULY',
        'LOCKKEEPERS',
        'MO MO`S KEBAB',
        'BAKE ME A WISH',
        'ONE POT CLEVELAND'
    ],
    'Fast Food': [
        'LEFTY',
        'SHAKE SHACK',
        'SHAKESHACK',
        'WENDY',
        'BUDDA',
        'CILANTRO',
        'PANERA',
        'CHIPOTLE',
        'BIBIBOP',
        'ROGERS',
        'PIADA',
        'ZINA',
        'SUBSHOPPE',
        'NATURES OASIS',
        'LOTUS EXPRESS',
        'BRUEGGERS '
    ],
    'World Food': [
        'KRAKOW',
        'NIPA HUT',
        'YELESEYEVSKY'
    ],
    'Why DOORDASH?': [
        'DOORDASH'
    ],
    'Aikido n Yoga': [
        'CHECK',
        ' YOGA'
    ],
    'Beauty n Supplies': [
        'LADIES',
        'BATH',
        'SALLY BEAUTY',
        'AVEDA',
        'LUSH BEACHWOOD',
        'AIKIKAI',
        'AIKIDO',
        'ATMA',
        'PADDLE',
        'OFFICEMAX'
    ],
    'Sound Bath': [
        'PAYPAL INST'
    ],
    'House Maintenance': [
        re.compile(r'HOME DEPOT.*CLEVELAND'),
        re.compile(r'HOME DEPOT.*OH$')
    ],
    'Furnishing': [
        'WORLD',
        'REFURNISHING',
        'KOALA',
        'WAYFAIR'
    ],
    'Garden': [
        'BREMEC',
        'LANDSCAPE',
        'STUMP',
        'NATURE CENTER'
    ],
    'Subscription': [
        'SPOTIFY',
        'APPLE',
        'NETFLIX',
        'AUDIBLE',
        'PEACOCK',
        'WALL',
        'BITDEFENDER',
        'MICROSOFT',
        'HULU',
        'NYTIMES',
        'IDEASTREAM',
        'WSJ'
    ],
    'Amazon': [
        'AMAZON',
        'AMZN'
    ],
    'Department Store': [
        'TARGET',
        'MACY'
    ],
    'Clothes': [
        'REI',
        'NORDSTROM',
        'DICK',
        'DSW',
        'AVALON',
        'MARSHALLS',
        'ANN TAYLOR',
        'AMERICAN EAGLE',
        'FOOTWEAR',
        'H&M ',
        'OLD NAVY ',
        "VICTORIA'S SECRET"
    ],
    'Kindle n Books': [
        'KINDLE',
        'AUDIOTEKA',
        'LOGANBERRY',
        "MAC'S",
        'EMPIK'
    ],
    'Software n Accessories': [
        'SIMON HAYNES',
        'ALISTORE',
        'GOOGLE',
        'FLIXEASY',
        'CLIP STUDIO',
        'ALIEXPRESS',
        'SERIF.COMBILL MINNETONKA MN'
    ],
    'Gifts': [
        'FIDDLEHEAD',
        'PASSPORT',
        'DIAMONDS FLOWERS',
        'BUNDT',
        'ALL CITY CANDY',
        'LOTUS FLOWER LLC',
        'PINKBLUSHMATERNIT',
        'LITTLE ROOM ',
        'CRAFT COLLECTIVE',
        'PANDORA',
        'WORDPRESS ',
        'STUFFEDANIMALSHOP',
        'GETSHIRTZ',
        'FLOWERS.COM',
        'PROGIFT',
        'BERKSHIRE BLANKET'
    ],
    'Kids Toys': [
        'PLAYMATTERS',
        'DISNEYSTORE',
        'CHILDRENS PLACE',
        'THECHILDRENSPLACE.COM'
    ],
    'Cycling n Paddling': [
        'BIKES',
        'IROCKER',
        'OHIO STATE PARKS'
    ],
    'Cat Food n Supplies': [
        'PET',
        'CHEWY',
        'HOLLYWOOD FEED',
        'JACKSON GALAXY'
    ],
    'Cat Health': [
        'VETERINARY'
    ],
    'Fun Out': [
        'CLEVELAND MUSEUM OF AR',
        'CHILDRENS MUSEUM ',
        'GREATER CLEVELAND AQUA',
        'METROPARKS FARMPA KIRTLAND',
        'MITCHELL',
        'SWEET FIX',
        'MANGO MANGO DESSE',
        'ON THE RISE',
        'RISING STAR COFFEE',
        'STARBUCKS',
        'MICHAELS',
        'UPTOWN MART',
        'ELLIE-MAYS',
        '6 FLAVORS INDIAN',
        'LUXE KITCHEN',
        'KOKO BAKERY'
    ],
    'Movies n Theater': [
        'VUDU',
        'FANDANGO',
        'DOBAMA',
        'THEAT',
        'CLEVELAND PUBLIC',
        'CLEVELAND INSTITUTE OF CLEVELAND',
        'MOVIE',
        'PRIME VIDEO',
        'BORDERLIGHT',
        'CINEMA'
    ],
    'Music n Games': [
        'STEAMGAMES',
        'BANDCAMP'
    ],
    'Memberships': [
        'Cleveland Museum Cleveland',
        'LITCLEVELAND'
    ],
    'Political Donations': [
        'ACTBLUE',
        'CUIMC '
    ],
    'ATM Wthdrw n Dpsit': [
        'ATM '
    ],
    'Vending Machines': [
        'VENDING ',
        'PEPSIVEN'
    ],
    'Parcels': [
        'USPS '
    ],
    'Air Travel': [
        'LOT ',
        'AMERICAN',
        'EXPEDIA',
        'SAS ',
        'EAT AND GO JAMAICA',
        ' Kastrup',
        'HUDSONNEWS ',
        'HUDSON',
        'HNDISCOVER',
        'JFK ',
        'MIDTOWN BISTRO',
        'TST* ',
        'CURRITO '
    ],
    'Visiting Grandma Ela': [
        'PLUSKI',
        'OLSZTYN',
        'WARSZAWA',
        re.compile(r' POL$'),
        re.compile(r' CHICAGO IL$')
    ],
    'Vacation SC': [
        'FOLLY',
        'VIATORTRIPADVISOR',
        'VACASA',
        'VRBO',
        re.compile(r' SC$'),
        re.compile(r' NC$'),
        re.compile(r' WV$')
    ],
    'Visiting Wanda': [
        re.compile(r' VA$'),
        re.compile(r' GA$'),
        re.compile(r' MD$')
    ],
    'Visiting Eva': [
        'PITTSBURGH',
        'TURNPIKE',
        re.compile(r' PA$')
    ]
}

In [ ]:
import pandas as pd

def build_categories_df(categories_to_subcategories_tree, subcategories_to_patterns):
    """
    Build a DataFrame from the categories and subcategories mapping.
    
    :param categories_to_subcategories_tree: Dictionary mapping categories to their subcategories.
    :param subcategories_to_patterns: Dictionary mapping subcategories to their patterns.
    :return: DataFrame with columns 'Category', 'Subcategory', 'Pattern', 'IsRegex'.
    """
    
    # Step 1: Create a mapping of subcategory → category
    subcategory_to_category = {
        subcat: category
        for category, subcats in categories_to_subcategories_tree.items()
        for subcat in subcats
    }

    # Step 2: Assemble the new categories_list
    categories_list = []

    for subcat, patterns in subcategories_to_patterns.items():
        category = subcategory_to_category.get(subcat, 'UNKNOWN')
        for pattern in patterns:
            pattern_text = pattern.pattern if hasattr(pattern, 'pattern') else pattern
            categories_list.append({
                'Category': category,
                'Subcategory': subcat,
                'Pattern': pattern_text,
                'IsRegex': hasattr(pattern, 'search')
            })

    return pd.DataFrame(categories_list)

In [ ]:
categories_df = build_categories_df(categories_to_subcategories_tree, subcategories_to_patterns)
print("Categories DataFrame sample:")
print(categories_df.head())

### 🔍 `categorize_transaction(description, categories_df) → tuple[str, str]`

Attempts to classify a single transaction description into a `(Category, Subcategory)` pair using pattern matching.

**Parameters:**
- `description` (`str`): Raw transaction description text.
- `categories_df` (`pd.DataFrame`): Classification table with columns:
  - `'Category'`: Top-level category name.
  - `'Subcategory'`: Subcategory name.
  - `'Pattern'`: Matching string or regex pattern.
  - `'IsRegex'`: Boolean flag indicating regex usage.

**Returns:**
- `tuple[str, str]`: A `(Category, Subcategory)` pair.
  - If a match is found, returns the corresponding category and subcategory.
  - If no match is found, returns `('UNCATEGORIZED', 'Uncategorized')`.

**Logic:**
1. Converts the input `description` to uppercase for case-insensitive matching.
2. Iterates over each row in `categories_df`.
3. Applies regex search if `IsRegex` is `True`; otherwise checks for substring presence.
4. Returns the first matching `(Category, Subcategory)` pair.
5. Defaults to `'UNCATEGORIZED'` if no match is found.

**Example:**
```python
categorize_transaction("Payment to ATT*BILL", categories_df)
# → ('HOUSING', 'Bills n Utilities')

In [ ]:
def categorize_transaction(description, categories_df):
    """
    ### 🔍 `categorize_transaction(description, categories_df) → tuple[str, str]`

    Attempts to classify a single transaction description into a `(Category, Subcategory)` pair using pattern matching.

    **Parameters:**
    - `description` (`str`): Raw transaction description text.
    - `categories_df` (`pd.DataFrame`): Classification table with columns:
    - `'Category'`: Top-level category name.
    - `'Subcategory'`: Subcategory name.
    - `'Pattern'`: Matching string or regex pattern.
    - `'IsRegex'`: Boolean flag indicating regex usage.

    **Returns:**
    - `tuple[str, str]`: A `(Category, Subcategory)` pair.
    - If a match is found, returns the corresponding category and subcategory.
    - If no match is found, returns `('UNCATEGORIZED', 'Uncategorized')`.

    **Logic:**
    1. Converts the input `description` to uppercase for case-insensitive matching.
    2. Iterates over each row in `categories_df`.
    3. Applies regex search if `IsRegex` is `True`; otherwise checks for substring presence.
    4. Returns the first matching `(Category, Subcategory)` pair.
    5. Defaults to `'UNCATEGORIZED'` if no match is found.

    **Example:**
    ```python
    categorize_transaction("Payment to ATT*BILL", categories_df)
    # → ('HOUSING', 'Bills n Utilities')"""
    description_upper = description.upper()

    for _, row in categories_df.iterrows():
        pattern = row['Pattern']
        if row['IsRegex']:
            if re.search(pattern, description_upper, flags=re.IGNORECASE):
                return row['Category'], row['Subcategory']
        else:
            if pattern.upper() in description_upper:
                return row['Category'], row['Subcategory']

    return 'UNCATEGORIZED', 'Uncategorized'

In [ ]:
def categorize_transactions(raw_transactions, categories_df):
    """
    Categorizes transactions in the DataFrame based on the provided categories DataFrame.
    
    Parameters:
    raw_transactions (DataFrame): DataFrame containing transactions with a 'Description' column.
    categories_df (DataFrame): DataFrame containing categories and subcategories with patterns.
    
    Returns:
    DataFrame: COPY OF THE Original DataFrame with two new columns: 'Category' and 'Subcategory'.
    """
    categorized_expenses_df = raw_transactions.copy()
    categorized_expenses_df[['Category', 'Subcategory']] = \
        raw_transactions['Description'].apply(lambda x: pd.Series(categorize_transaction(x, categories_df)))
    return categorized_expenses_df


In [ ]:
def categorize_transactions_with_annotate(raw_transactions, categories_df):
    """
    Categorizes transactions in the DataFrame based on the provided categories DataFrame,
    with optional override via 'Annotation' and passthrough of 'Comments'.

    Parameters:
    raw_transactions (DataFrame): DataFrame containing transactions with a 'Description' column.
                                  Optionally includes 'Annotation' and 'Comments' columns.
    categories_df (DataFrame): DataFrame containing categories and subcategories with patterns.

    Returns:
    DataFrame: Copy of the original DataFrame with new columns: 'Category', 'Subcategory', 'Comments'.
    """
    categorized_expenses_df = raw_transactions.copy()

    # Build subcategory → category lookup
    subcategory_to_category = {
        row['Subcategory']: row['Category']
        for _, row in categories_df.iterrows()
    }

    def resolve_transaction(row):
        # Check for manual override via Annotation
        annotation = row.get('Annotation')
        if pd.notna(annotation) and annotation in subcategory_to_category:
            return pd.Series({
                'Category': subcategory_to_category[annotation],
                'Subcategory': annotation
            })
        # Fallback to pattern matching
        return pd.Series(categorize_transaction(row['Description'], categories_df))

    # Apply categorization logic
    categorized_expenses_df[['Category', 'Subcategory']] = raw_transactions.apply(resolve_transaction, axis=1)

    # Copy Comments if present
    if 'Comments' in raw_transactions.columns:
        categorized_expenses_df['Comments'] = raw_transactions['Comments']

    return categorized_expenses_df

In [ ]:
# Load a pair of bank record files for a particular year and month 
# NOTE - uses standard file naming convention for checking and credit card record  
def load_expenses(year_month):
    # Load the CSV files with credit card and checking card records
    creditcard_filename = 'data/' + year_month + '-A&T-CCard.xlsx'
    checking_filename = 'data/' + year_month + '-A&T-CheckingAcct.xlsx'

    df_credit = pd.read_excel(creditcard_filename)
    df_checking = pd.read_excel(checking_filename)
    print(df_credit.head())
    print(df_checking.head())

    # Check the first few rows to understand the structure of your data.
    df = pd.concat([df_checking, df_credit], ignore_index=True)
    df.head()
    return df

In [ ]:
# set pandas display options to show all columns
pd.set_option("display.width", 1000)
pd.set_option("display.max_columns", None)  # Show all columns

In [ ]:
# year_month = '2025-01'  # Example month
# df_monthly_expenses = load_expenses(year_month)
# df_monthly_expenses.head()

In [ ]:
# categories_df = build_categories_df(categories_to_subcategories_tree, subcategories_to_patterns)
# print(categories_df)
# categorized_expenses_df = categorize_transactions(df_monthly_expenses, categories_df)
# print(categories_df)
# print(categorized_expenses_df.head())
# print(categorized_expenses_df[['Description', 'Subcategory', 'Category']].head(10))  # first 10 rows

In [ ]:
def show_uncategorized_transactions(categorized_expenses_df):
    """
    Show uncategorized transactions from the categorized DataFrame.
    
    Parameters:
    categorized_expenses_df (DataFrame): DataFrame with categorized transactions.
    
    Returns:
    DataFrame: DataFrame containing uncategorized transactions.
    """
    uncategorized = categorized_expenses_df[categorized_expenses_df['Category'] == 'UNCATEGORIZED']
    print(f"{len(uncategorized)} uncategorized transactions")
    return uncategorized[['Date', 'Amount', 'Description']]

In [ ]:
def correct_date_format(categorized_expenses_df):
    """
    Correct the date format in the categorized expenses DataFrame.
    
    Parameters:
    categorized_expenses_df (DataFrame): DataFrame with a 'Date' column to be corrected.
    
    Returns:
    None: The DataFrame is modified in place.
    """
    # Convert 'Date' column to datetime format and then to string in 'YYYY-MM-DD' format
    categorized_expenses_df['Date'] = pd.to_datetime(categorized_expenses_df['Date']).dt.strftime('%Y-%m-%d')
    categorized_expenses_df['Date'] = pd.to_datetime(categorized_expenses_df['Date'])

In [ ]:
# Use of ExcelWriter to manage Excel formatting and saving workbook and sheet
def UseExcelWriter(categorized_expenses_df, year_month):
    """
    Use of ExcelWriter to manage Excel formatting and saving workbook and sheet.
    """
    output_path = 'categorized_transactions' + year_month + '_formatted.xlsx'
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        # Write your DataFrame
        categorized_expenses_df.to_excel(writer, sheet_name='Expenses', index=False)

        # Access Excel components
        workbook = writer.book
        worksheet = writer.sheets['Expenses']

        # Apply formatting to a column (e.g. Date)
        date_format = workbook.add_format({'num_format': 'yyyy-mm-dd'})
        worksheet.set_column('A:A', 15, date_format)  # Assuming column A is 'Date'

    # ✅ File is written and closed when the 'with' block ends

In [ ]:
def save_to_excel(categorized_expenses_df, year_month, long_format=False):
    """
    Save the categorized expenses DataFrame to an Excel file.
    
    Parameters:
    categorized_expenses_df (DataFrame): DataFrame with categorized expenses.
    year_month (str): Year and month string to include in the filename.
    
    Returns:
    None: The DataFrame is saved to an Excel file.
    """

    if long_format:
        # Save in long format
        output_path = 'categorized_transactions' + year_month + '_filtered.xlsx'
        categorized_expenses_df[['Date', 'Description', 'Amount', 'Subcategory', 'Category']].to_excel(output_path, index=False)
    else:
        output_path = 'categorized_transactions' + year_month + '.xlsx'
        # format the 'Date' column to 'YYYY-MM-DD' and save to Excel
        categorized_expenses_df['Date'] = pd.to_datetime(categorized_expenses_df['Date']).dt.strftime('%Y-%m-%d')
        categorized_expenses_df.to_excel(output_path, index=False)\

    print(f"✅ Saved categorized results to: {output_path}")

In [ ]:
import pandas as pd
import re
from datetime import datetime

class MonthlyExpenseReport:
    def __init__(self, year_month: str):
        """
        Initialize with a raw transaction DataFrame and metadata
        """
        self.year_month = year_month  # format: 'yy-mm'
        self.categories_df =  build_categories_df(categories_to_subcategories_tree, subcategories_to_patterns)
        self.transactions_raw = load_expenses(self.year_month)
        self.transactions_categorized = None
        self.summary_table = None

    def read_bank_records(self):
        """ read bank records for a given month"""
        self.transactions_raw = load_expenses(self.year_month)  

    def preprocess(self):
        """Ensure proper datetime format and extract YearMonth label"""
        self.transactions_raw['Date'] = pd.to_datetime(self.transactions_raw['Date'])
        # self.transactions_raw['YearMonth'] = self.transactions_raw['Date'].dt.strftime('%y-%m')

    def categorize_transactions(self):
        """Apply pattern matching and build categorized DataFrame"""
        def _categorize_row(row):
            desc = row['Description']
            for _, rule in self.categories_df.iterrows():
                pat = rule['Pattern']
                if rule['IsRegex']:
                    if re.search(pat, desc, flags=re.IGNORECASE):
                        return pd.Series([rule['Subcategory'], rule['Category']])
                else:
                    if pat.upper() in desc.upper():
                        return pd.Series([rule['Subcategory'], rule['Category']])
            return pd.Series(['Uncategorized', 'UNCATEGORIZED'])

        df = self.transactions_raw.copy()
        df[['Subcategory', 'Category']] = df.apply(_categorize_row, axis=1)
        self.transactions_categorized = df

    def categorize_transactions_with_annotate(self):
        """Categorize transactions using Annotation override and pattern matching on Description."""

        # Build subcategory → category lookup
        subcategory_to_category = {
            row['Subcategory']: row['Category']
            for _, row in self.categories_df.iterrows()
        }

        def _categorize_row(row):
            # Manual override via Annotation
            annotation = row.get('Annotation')
            if pd.notna(annotation) and annotation in subcategory_to_category:
                return pd.Series({
                    'Subcategory': annotation,
                    'Category': subcategory_to_category[annotation]
                })

            # Fallback to pattern matching on Description
            desc = row.get('Description', '')
            for _, rule in self.categories_df.iterrows():
                pat = rule['Pattern']
                if rule['IsRegex']:
                    if re.search(pat, desc, flags=re.IGNORECASE):
                        return pd.Series({
                            'Subcategory': rule['Subcategory'],
                            'Category': rule['Category']
                        })
                else:
                    if pat.upper() in desc.upper():
                        return pd.Series({
                            'Subcategory': rule['Subcategory'],
                            'Category': rule['Category']
                        })

            return pd.Series({'Subcategory': 'Uncategorized', 'Category': 'UNCATEGORIZED'})

        # Apply categorization
        df = self.transactions_raw.copy()
        df[['Subcategory', 'Category']] = df.apply(_categorize_row, axis=1)

        # Preserve Comments if present
        if 'Comments' in self.transactions_raw.columns:
            df['Comments'] = self.transactions_raw['Comments']

        self.transactions_categorized = df

    def summarize(self):
        """Creates a cleanly indexed summary table with real category + subcategory labels and correct totals."""

        # Ordered structure from category tree
        cat_sub_list = [
            (category, subcat)
            for category, subcats in categories_to_subcategories_tree.items()
            for subcat in subcats
        ]
        cat_sub_index = pd.MultiIndex.from_tuples(
            cat_sub_list, names=['Category', 'Subcategory']
        )
        print("cat_sub_index sample:", cat_sub_index.tolist()[:10])

        # Group actual expenses by category/subcategory (returns a Series!)
        grouped_series = (
            self.transactions_categorized
            .groupby(['Category', 'Subcategory'])['Amount']
            .sum()
        )

        # Convert to DataFrame explicitly and reindex
        grouped_df = grouped_series.to_frame(name='Amount') \
            .reindex(cat_sub_index, fill_value=0) \
            .reset_index()

        # Final column cleanup
        grouped_df.rename(columns={
            'Category': 'CATEGORIES',
            'Subcategory': 'SUBCATEGORIES',
            'Amount': 'AMOUNT'
        }, inplace=True)

        # ... after grouped_df is created and columns are renamed ...
        ordered_index = pd.MultiIndex.from_tuples(
            [
                (cat, subcat)
                for cat, subcats in categories_to_subcategories_tree.items()
                for subcat in subcats
            ],
            names=['CATEGORIES', 'SUBCATEGORIES']
        )
        # Enforce semantic order
        self.summary_table = (
            grouped_df
            .set_index(['CATEGORIES', 'SUBCATEGORIES'])
            .reindex(ordered_index, fill_value=0)
            .reset_index()
        )

        self.summary_table = grouped_df

    def export_summary_excel(self, output_path: str):
        """Save summary table with clean formatting and custom sheet name"""
        sheet_name = f"Summary_{self.year_month}"

        print("Summary table:\n", self.summary_table.head(10))
        print("Columns:", self.summary_table.columns.tolist())

        with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
            self.summary_table.to_excel(writer, sheet_name=sheet_name, index=False)

            workbook = writer.book
            worksheet = writer.sheets[sheet_name]

            # Format columns
            header_format = workbook.add_format({'bold': True})
            money_format = workbook.add_format({'num_format': '$#,##0.00'})

            # Set column widths and formats by header
            worksheet.set_column('A:A', 22, None)         # CATEGORIES
            worksheet.set_column('B:B', 26, None)         # SUBCATEGORIES
            worksheet.set_column('C:C', 15, money_format) # AMOUNT

            # Bold headers (optional, decorative)
            for col_num, value in enumerate(self.summary_table.columns):
                worksheet.write(0, col_num, value, header_format)

    def export_transactions_excel(self, output_path: str):
        """Export full transaction list with formatted dates"""
        df = self.transactions_categorized.copy()
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        df.to_excel(output_path, index=False, sheet_name=f'Tnxs_{self.year_month}')

    def run_full_pipeline(self, use_annotation: bool=False):
        """Convenience method"""
        self.preprocess()
        if use_annotation:
            self.categorize_transactions_with_annotate()
        else: 
            self.categorize_transactions()
        self.summarize()

In [ ]:
class MultiMonthExpenseWorkbook:
    def __init__(self, reports: list[MonthlyExpenseReport]):
        self.reports = reports
        self.summary_pivot = None

    def enforce_semantic_order(self):
        ordered_pairs = [
            (cat, subcat)
            for cat, subcats in categories_to_subcategories_tree.items()
            for subcat in subcats
        ]

        # Reindex the merged summary_pivot
        self.summary_pivot = (
            self.summary_pivot
            .set_index(['CATEGORIES', 'SUBCATEGORIES'])
            .reindex(ordered_pairs, fill_value=0)
            .reset_index()
        )

        # Ensure the columns are in the correct order
        # Capture fixed columns
        fixed_cols = ['CATEGORIES', 'SUBCATEGORIES']
        # Extract and sort month columns
        month_cols = sorted(
            [col for col in self.summary_pivot.columns if col not in fixed_cols]
        )
        # Reorder DataFrame
        self.summary_pivot = self.summary_pivot[fixed_cols + month_cols]        

        
    def build_combined_summary(self):
        """Pivot all monthly summaries into a single table."""
        summary_frames = []

        for report in self.reports:
            df = report.summary_table.copy()
            df = df.rename(columns={'AMOUNT': report.year_month})
            summary_frames.append(df)

        merged = summary_frames[0][['CATEGORIES', 'SUBCATEGORIES', self.reports[0].year_month]]

        for df in summary_frames[1:]:
            merged = merged.merge(df, on=['CATEGORIES', 'SUBCATEGORIES'], how='outer')

        merged.fillna(0, inplace=True)
        self.summary_pivot = merged

    def export_all_to_excel(self, output_path: str):
        """Save summary and all transaction sheets to a single workbook."""
        self.build_combined_summary()
        self.enforce_semantic_order()

        with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
            # Write the combined summary first
            self.summary_pivot.to_excel(writer, sheet_name='Monthly Summary', index=False)

            # Format summary
            workbook = writer.book
            summary_ws = writer.sheets['Monthly Summary']
            currency_fmt = workbook.add_format({'num_format': '$#,##0.00'})
            summary_ws.set_column('A:B', 22)  # Categories/Subcategories
            summary_ws.set_column(2, 1 + len(self.reports), 14, currency_fmt)

            # Write each month's transactions sheet
            for report in self.reports:
                sheet_name = f"Txns_{report.year_month}"
                df = report.transactions_categorized.copy()
                df = df.sort_values('Date')  # ← this ensures chronological order
                df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
                df.to_excel(writer, sheet_name=sheet_name, index=False)

                # Optional formatting for txn sheets
                tx_ws = writer.sheets[sheet_name]
                tx_ws.set_column('A:A', 14)  # Date
                tx_ws.set_column('B:B', 40)  # Description
                tx_ws.set_column('C:D', 15)  # Amount + Type

                tx_ws.freeze_panes(1, 0)  # Keeps header row visible while scrolling
                tx_ws.autofilter(0, 0, df.shape[0], df.shape[1] - 1)  # Enables filtering on all columns

                # Dynamically adjust each column width based on content
                for i, col in enumerate(df.columns):
                    # Get max width between header and the longest string in the column
                    max_len = max(
                        df[col].astype(str).map(len).max(),
                        len(str(col))
                    ) + 2  # optional padding for readability

                    tx_ws.set_column(i, i, max_len)

In [ ]:
month_label = '2025-08'

# jan_df = load_expenses(month_label)
# df_monthly_expenses.head()

report = MonthlyExpenseReport(year_month=month_label)
report.run_full_pipeline(use_annotation=False)

# Save results
report.export_transactions_excel(f'categorized_transactions_{month_label}.xlsx')
report.export_summary_excel(f'monthly_summary_{month_label}.xlsx')

annotated_report = MonthlyExpenseReport(year_month=month_label)
annotated_report.run_full_pipeline(use_annotation=True)

# Save results
annotated_report.export_transactions_excel(f'categorized_transactions_{month_label}_annotated.xlsx')
annotated_report.export_summary_excel(f'monthly_summary_{month_label}_annotated.xlsx')

In [ ]:
# Jan_25_label = '2025-01'
# Feb_25_label = '2025-02'
# Mar_25_label = '2025-03'
# Apr_25_label = '2025-04'

# jan_report = MonthlyExpenseReport(year_month=Jan_25_label)
# feb_report = MonthlyExpenseReport(year_month=Feb_25_label)
# mar_report = MonthlyExpenseReport(year_month=Mar_25_label)
# apr_report = MonthlyExpenseReport(year_month=Apr_25_label)

# all_reports = [jan_report, feb_report, mar_report]
# book = MultiMonthExpenseWorkbook(all_reports)
# book.export_all_to_excel("q1_expenses_summary.xlsx")

In [ ]:
month_labels = {
    '2025-01',
    '2025-02',
    '2025-03',
    '2025-04',
    '2025-05',
    '2025-06',
    '2025-07',
    '2025-08'
}
summary_label = '2025-01_to_08'

reports = []

for month_label in month_labels:
    report = MonthlyExpenseReport(year_month=month_label)
    report.run_full_pipeline()  # ← This processes everything
    reports.append(report)

In [ ]:
book = MultiMonthExpenseWorkbook(reports)
book.export_all_to_excel(f'quarterly_expense_summary_{summary_label}.xlsx')